In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [25]:
class Expando(object):
    '''
        A small class which can have attributes set
    '''
    pass


class InputData:
    
    def __init__(
        self, 
        hours: dict[str, float],
        wind_forecast: dict[str, float],
        wind_realization: dict[str, float],
        price_da: dict[str, float],
        price_bal_up: dict[str, float],
        price_bal_down: dict[str, float],
        nameplate_capacity: float,
    ):
        self.hours = hours
        self.wind_forecast = wind_forecast
        self.wind_realization = wind_realization
        self.price_da = price_da
        self.price_bal_up = price_bal_up
        self.price_bal_down = price_bal_down
        self.nameplate_capacity = nameplate_capacity


class Bid_DA():

    def __init__(self, input_data: InputData):
        self.data = input_data 
        self.variables = Expando()
        self.constraints = Expando() 
        self.results = Expando() 
        self._build_model() 

    def _build_variables(self):
        self.variables.bid_DA = {
            h: self.model.addVar(
                lb=0, ub=GRB.INFINITY, name='Bid in day-ahead market'
            ) for h in self.data.hours
            }
        
        self.variables.delta = {
            h: self.model.addVar(
                lb=-GRB.INFINITY, ub=GRB.INFINITY, name='Difference between bid and realization'
            ) for h in self.data.hours
            }

        self.variables.delta_up = {
            h: self.model.addVar(
                lb=0, ub=GRB.INFINITY, name='Positive difference between bid and realization'
            ) for h in self.data.hours
            }
        
        self.variables.delta_down = {
            h: self.model.addVar(
                lb=0, ub=GRB.INFINITY, name='Negative difference between bid and realization'
            ) for h in self.data.hours
            }
        
    
    def _build_constraints(self):
        self.constraints.bid_DA = {
            h: self.model.addConstr(
                self.variables.bid_DA[h] <= self.data.nameplate_capacity, 
                name='Bid in day-ahead market should be less than or equal to nameplate capacity'
            ) for h in self.data.hours
            }
        
        self.constraints.delta = {
            h: self.model.addConstr(
                self.variables.delta[h] == self.variables.bid_DA[h] - self.data.wind_realization[h], 
                name='Difference between bid and realization'
            ) for h in self.data.hours
            }

        self.constraints.delta_up = {
            h: self.model.addConstr(
                self.variables.delta[h] == self.variables.delta_up[h] - self.variables.delta_down[h],
            ) for h in self.data.hours
            }


    def _build_objective_function(self):
        
        objective = (
            gp.quicksum(
                self.data.price_da[h] * self.variables.bid_DA[h] + 
                self.data.price_bal_up[h] * self.variables.delta_up[h] - 
                self.data.price_bal_down[h] * self.variables.delta_down[h]
                for h in self.data.hours
            )
        )
        self.model.setObjective(objective, GRB.MAXIMIZE)

    def _build_model(self):
        self.model = gp.Model(name='Optimized DA bid for 24 hours')
        self._build_variables()
        self._build_constraints()
        self._build_objective_function()
        self.model.update()

    def _save_results(self):
        self.results.bid_DA = {
            h: self.variables.bid_DA[h].x for h in self.data.hours
        }
        self.results.delta = {
            h: self.variables.delta[h].x for h in self.data.hours
        }
        self.results.profit_DA = {
            h: self.data.price_da[h] * self.variables.bid_DA[h].x for h in self.data.hours
        }
        self.results.profit_bal_up = {
            h: self.data.price_bal_up[h] * self.variables.delta_up[h].x for h in self.data.hours
        }
        self.results.total_profit = {
            h: self.results.profit_DA[h] + self.results.profit_bal_up[h] for h in self.data.hours
        }
        results = {
            'Hour': list(self.data.hours),
            'Bid_DA': [self.results.bid_DA[h] for h in self.data.hours],
            'Delta': [self.results.delta[h] for h in self.data.hours],
            'Profit_DA': [self.results.profit_DA[h] for h in self.data.hours],
            'Profit_Bal_Up': [self.results.profit_bal_up[h] for h in self.data.hours],
            'Total_Profit': [self.results.total_profit[h] for h in self.data.hours],
        }

        # Create a DataFrame using this reorganized data
        self.results.df = pd.DataFrame(results)

        


    def run(self):
        self.model.optimize()
        if self.model.status == GRB.OPTIMAL:
            self._save_results()
        else:
            raise RuntimeError(f"optimization was not successful")
    
    def display_results(self):
        print()
        print("-------------------   RESULTS  -------------------")
        print("Detailled results:")
        print(self.results.df)
        print("--------------------------------------------------")

    
   


In [26]:
# create an input data object with random data for 24 hours
input_data = InputData(
    hours = [str(i) for i in range(24)],
    wind_forecast = {str(i): 100 for i in range(24)},
    wind_realization = {str(i): 100 for i in range(24)},
    price_da = {str(i): 10 for i in range(24)},
    price_bal_up = {str(i): 10 for i in range(24)},
    price_bal_down = {str(i): 10 for i in range(24)},
    nameplate_capacity = 1000
)
    

In [27]:
model = Bid_DA(input_data)
model.run()
model.display_results()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 72 rows, 96 columns and 144 nonzeros
Model fingerprint: 0x9400b946
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+03]
Presolve removed 72 rows and 96 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5600000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.560000000e+05

-------------------   RESULTS  -------------------
Detailled results:
   Hour  Bid_DA  Delta  Profit_DA  Profit_Bal_Up  Total_Profit
0     0  1000.0  900.0    10000.0         9000.0       19000.0
1     1  1000.0 